In [1]:
import getpass
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("What is the weather in Caracas, Venezuela")
print(search_results)
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Caracas', 'region': 'Distrito Federal', 'country': 'Venezuela', 'lat': 10.5, 'lon': -66.9167, 'tz_id': 'America/Caracas', 'localtime_epoch': 1741062415, 'localtime': '2025-03-04 00:26'}, 'current': {'last_updated_epoch': 1741061700, 'last_updated': '2025-03-04 00:15', 'temp_c': 25.1, 'temp_f': 77.2, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 156, 'wind_dir': 'SSE', 'pressure_mb': 1014.0, 'pressure_in': 29.94, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 74, 'cloud': 0, 'feelslike_c': 27.5, 'feelslike_f': 81.5, 'windchill_c': 17.5, 'windchill_f': 63.5, 'heatindex_c': 17.5, 'heatindex_f': 63.5, 'dewpoint_c': 14.8, 'dewpoint_f': 58.7, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 6.7, 'gust_kph': 10.8}}"}, {'url': 'https://weather.com/weather/hourbyhour/l/Caracas+Capital+Distric

In [3]:
from langchain.chat_models import init_chat_model

model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")


In [4]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="How is the weather usually in Canada?")])
print(response.content)

Canada's weather is diverse and varies greatly depending on the region and season. The country's vast size and geography, which includes mountains, forests, and coastlines, contribute to its complex climate. Here's a general overview of Canada's weather patterns:

**Seasons:**

1. **Winter (December to February):** Cold and snowy, with average temperatures ranging from -2°C to -15°C (28°F to 5°F) in the south and -20°C to -30°C (-4°F to -22°F) in the north.
2. **Spring (March to May):** Mild and unpredictable, with temperatures gradually warming up from -5°C to 15°C (23°F to 59°F).
3. **Summer (June to August):** Warm and humid, with average temperatures ranging from 15°C to 25°C (59°F to 77°F) in the south and 10°C to 20°C (50°F to 68°F) in the north.
4. **Autumn (September to November):** Cool and colorful, with temperatures gradually cooling down from 10°C to 20°C (50°F to 68°F).

**Regional Weather Patterns:**

1. **Pacific Coast (British Columbia):** Mild and wet, with significant

In [5]:
model_with_tools = model.bind_tools(tools)

In [6]:
response2 = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response2.content}")
print(f"ToolCalls: {response2.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather today'}, 'id': 'call_hgkh', 'type': 'tool_call'}]


In [7]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [8]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools, checkpointer=memory)
config = {"configurable" : { "thread_id" : "abc123"}}

In [9]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hola. Me llamo Carlos, Me pudieras decir que tan seguro es un tesla cybertruck?")]}, 
    config,
    stream_mode="values",
):
    print(chunk)
    chunk["messages"][-1].pretty_print()
    print("----")

{'messages': [HumanMessage(content='hola. Me llamo Carlos, Me pudieras decir que tan seguro es un tesla cybertruck?', additional_kwargs={}, response_metadata={}, id='b48822e1-9972-4b7e-b901-477161cb1aeb')]}
================================ Human Message =================================

hola. Me llamo Carlos, Me pudieras decir que tan seguro es un tesla cybertruck?
----
{'messages': [HumanMessage(content='hola. Me llamo Carlos, Me pudieras decir que tan seguro es un tesla cybertruck?', additional_kwargs={}, response_metadata={}, id='b48822e1-9972-4b7e-b901-477161cb1aeb'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_en4w', 'function': {'arguments': '{"query": "Tesla Cybertruck safety features"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 301, 'total_tokens': 323, 'completion_time': 0.08, 'prompt_time': 0.013894832, 'queue_time': 0.198525518, 'total_time': 0.09389483

In [10]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Me puedes decir como me llamo?")]}, 
    config,
    stream_mode="values",
):
    print(chunk)
    chunk["messages"][-1].pretty_print()
    print("----")

{'messages': [HumanMessage(content='hola. Me llamo Carlos, Me pudieras decir que tan seguro es un tesla cybertruck?', additional_kwargs={}, response_metadata={}, id='b48822e1-9972-4b7e-b901-477161cb1aeb'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_en4w', 'function': {'arguments': '{"query": "Tesla Cybertruck safety features"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 301, 'total_tokens': 323, 'completion_time': 0.08, 'prompt_time': 0.013894832, 'queue_time': 0.198525518, 'total_time': 0.093894832}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ca0059abb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6c796e92-1499-489b-b1c2-996f5cc5661c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Tesla Cybertruck safety features'}, 'id': 'call_en4w', 'type': 'tool_call'}], usage_metadata={'input_tokens': 301, 'outpu